In [1]:
!nvidia-smi

Thu Jun  2 14:42:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/My\ Drive/ /mydrive

## Clone and configure Darknet
---

In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15420, done.
remote: Total 15420 (delta 0), reused 0 (delta 0), pack-reused 15420
Receiving objects: 100% (15420/15420), 14.05 MiB | 23.99 MiB/s, done.
Resolving deltas: 100% (10360/10360), done.


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

## Configure YOLOv3 training parameters
---


In [6]:
!cp /content/darknet/cfg/yolov3.cfg /content/darknet/cfg/yolov3_training.cfg

In [7]:
!sed -i 's/max_batches = 500200/max_batches = 8000/' /content/darknet/cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=4@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=4@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=4@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=27@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=27@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=27@' /content/darknet/cfg/yolov3_training.cfg

In [8]:
!echo -e 'holothurian\nechinus\nscallop\nstarfish' > data/obj.names

In [ ]:
yolo_weights = 'yolov3_normal' #yolov3_normal or yolov3_2000 or yolov3_6000
yolo_test_set = 'yolo_test.zip' #yolo_test or eyolo_2000_test or eyolo_6000_test

In [9]:
# Change backup folder as necessary
!echo -e 'classes=4\ntrain=data/train.txt\nvalid=data/test.txt\nnames=data/obj.names\nbackup=/mydrive/yolov3_normal' > data/obj.data

## Get test dataset
---


In [11]:
# Change dataset folder as necessary 
!unzip -q /mydrive/yolo_test.zip -d data/testobj

In [12]:
import glob
test_images_list = glob.glob("data/testobj/*.jpg") #Ensure path to each image is correct
with open("data/test.txt", "w") as f:
    f.write("\n".join(test_images_list))

## Testing
---

In [ ]:
# To see predictions for a single image (output image saved as 'darknet/predictions.txt')
# Can change: weights folder (Ex: yolov3_normal), weights file (Ex: yolov3_training_7000.weights), image (Ex: 100.jpg)
!./darknet detector test data/obj.data cfg/yolov3_training.cfg /content/gdrive/MyDrive/yolov3_normal/yolov3_training_7000.weights /content/darknet/data/testobj/666.jpg

In [ ]:
# To generate mAP score and other metrics (saved as specified .txt file)
!./darknet detector map data/obj.data cfg/yolov3_training.cfg /content/gdrive/MyDrive/yolov3_normal/yolov3_training_7000.weights -dont_show < data/test.txt > noGAN_map_7000.txt

## Draw ground truth bounding box
---

In [4]:
import json
import pandas as pd
from google.colab.patches import cv2
import matplotlib.pyplot as plt
import math
import numpy as np
from PIL import Image, ImageDraw, ImageFont

In [5]:
# Change path to DUO dataset zip file as necessary
!unzip -q /content/gdrive/MyDrive/FYP/DATASETS/DUO.zip -d /content

In [6]:
classes_file = open('/content/DUO/annotations/instances_test.json')
classes_dict = json.load(classes_file)
class_ids = []
class_names = []
for i in classes_dict['categories']:
  class_ids.append(i['id'])
  class_names.append(i['name'])

In [7]:
image_annotations = {}
for i in classes_dict['annotations']:
  if i["image_id"] in image_annotations:
    image_annotations[i["image_id"]] = image_annotations[i["image_id"]] + [i]
  else:
    image_annotations[i["image_id"]] = [i]

In [8]:
# Colors of bounding box of each class
colors = [(255, 0, 255), (255,0,0), (50,205,50), (0, 255, 255)]

In [9]:
# Load specified image
def load_image(img_num):
    img = "/content/DUO/images/test/"+str(img_num)+".jpg"
    img = Image.open(img)
    return img

In [10]:
# Display an image
def display_image(img):
    imgplot = plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:
img_num = 100 # Select image

img_anns = image_annotations[img_num]

img = load_image(img_num)
canvas = ImageDraw.Draw(img)

# Drawing bounding boxes
for ann in img_anns:
  bbox = ann["bbox"]
  class_id = ann["category_id"]
  class_name = class_names[class_id-1]

  x_min = bbox[0]
  y_min = bbox[1]
  x_max = bbox[0]+bbox[2]
  y_max = bbox[1]+bbox[3]

  shape = [(x_min, y_min), (x_max, y_max)]
  text_pos = (x_min,y_min-18)

  # IMPORTANT: upload font file!
  # Change font size and width of rectangle as necessary
  font = ImageFont.truetype("/content/OpenSans-Regular.ttf", 15)

  canvas.rectangle(shape, outline=colors[class_id-1], width=3)
  canvas.text(text_pos, class_name, fill=colors[class_id-1], font=font)

# Save image for download (or can save directly to Drive if required)
img.save('/content/bb_drawn.png')
# Display image with bounding box annotations
display_image(img)